In [1]:
import pickle

import numpy as np
import pandas as pd

from cyclops.processors.aggregate import Aggregator
from cyclops.processors.cleaning import (
    normalize_categories,
    normalize_names,
    normalize_values,
)
from cyclops.processors.column_names import (
    ADMIT_TIMESTAMP,
    AGE,
    CARE_UNIT,
    DIAGNOSIS_CODE,
    DIAGNOSIS_TRAJECTORY,
    ENCOUNTER_ID,
    EVENT_CATEGORY,
    EVENT_NAME,
    EVENT_TIMESTAMP,
    EVENT_VALUE,
    EVENT_VALUE_UNIT,
    HOSPITAL_ID,
    SEX,
    SUBJECT_ID,
    TIMESTEP,
    YEAR,
)
from cyclops.processors.constants import (
    BINARY,
    BY,
    CATEGORICAL_INDICATOR,
    FEATURE_INDICATOR_ATTR,
    FEATURE_MAPPING_ATTR,
    FEATURE_TYPE_ATTR,
    FEATURE_TYPES,
    FEATURES,
    MEAN,
    MIN_MAX,
    MISSING_CATEGORY,
    NUMERIC,
    ORDINAL,
    STANDARD,
    STRING,
    TARGETS,
)
from cyclops.processors.feature.feature import TabularFeatures, TemporalFeatures
from cyclops.processors.feature.normalization import VectorizedNormalizer
from cyclops.processors.feature.split import split_idx
from cyclops.processors.feature.vectorize import intersect_vectorized, split_vectorized
from cyclops.query import mimic
from cyclops.query import process as qp
from cyclops.utils.common import print_dict
from cyclops.utils.file import load_array, load_dataframe, save_dataframe

2022-07-18 15:27:59,421 INFO cyclops.orm     - Database setup, ready to run queries!


In [2]:
YEARS = [2015, 2016, 2018, 2019, 2020]
MIN_YEAR = min(YEARS)
TARGET_NAME = "death"
TARGET_NAME_PROCESSED = "targets - death"

# Querying

## Patient encounters

In [ ]:
encounters_interface = mimic.patient_encounters(
    years=YEARS, died=True, died_binarize_col="died"
)
encounters_query = encounters_interface.query
encounters_query = qp.Drop(
    ["insurance", "language", "marital_status", "hospital_expire_flag"]
)(encounters_query)
encounters_interface = mimic.get_interface(encounters_query)
encounters = encounters_interface.run()
encounters

In [ ]:
save_dataframe(encounters, "encounters.parquet")

## Events

In [ ]:
events_interface = mimic.events()
events_query = events_interface.query
events_query = qp.Drop(["warning", "itemid", "storetime"])(events_query)
events_interface = mimic.get_interface(events_query)
events = events_interface.run(limit=1000000)

# Reverse the deidentified dating
events = pd.merge(
    encounters[[ENCOUNTER_ID, "anchor_year_difference"]], events, on=ENCOUNTER_ID
)


def add_offset(row):
    row[EVENT_TIMESTAMP] += pd.DateOffset(years=row["anchor_year_difference"])
    return row


events = events.apply(add_offset, axis=1)
events = events.drop("anchor_year_difference", axis=1)

In [ ]:
# Create the target as a timeseries event
target_events = encounters[encounters["died"] == True]
target_events = target_events[[ENCOUNTER_ID, "deathtime"]]
target_events = target_events.rename({"deathtime": EVENT_TIMESTAMP}, axis=1)
target_events[EVENT_NAME] = TARGET_NAME
target_events[EVENT_CATEGORY] = TARGETS
target_events[EVENT_VALUE] = 1
target_events.head(5)

In [ ]:
# Include target
events = pd.concat([events, target_events])

In [ ]:
# Preprocessing
events[EVENT_NAME] = normalize_names(events[EVENT_NAME])
events[EVENT_CATEGORY] = normalize_categories(events[EVENT_CATEGORY])
# events[EVENT_VALUE] = normalize_values(events[EVENT_VALUE])

# Concatenate event name and category since some names are the same in
# different categories, e.g., 'flow' for categories 'heartware' and 'ecmo'
events[EVENT_NAME] = events[EVENT_CATEGORY] + " - " + events[EVENT_NAME]
events.head(5)

# Update target name having included the category
TARGET_NAME = TARGETS + " - " + TARGET_NAME

In [ ]:
save_dataframe(events, "events.parquet")

------------------------------------------------------------------------------------------------

# Processing

## Tabular

In [3]:
encounters = load_dataframe("encounters.parquet")
encounters.head(5)

2022-07-18 15:28:01,238 INFO cyclops.utils.file - Loading DataFrame from encounters.parquet


,subject_id,encounter_id,admit_timestamp,discharge_timestamp,deathtime,admission_type,admission_location,discharge_location,ethnicity,edregtime,edouttime,sex,age,birth_year,dod,anchor_year_difference,died
0,11771268,27824037,2015-07-06 18:21:00,2015-07-07 20:15:00,NaT,ELECTIVE,None,HOME,UNKNOWN,NaT,NaT,M,0,2015,NaT,-156,False
1,17978591,21300810,2015-10-10 00:21:00,2015-10-12 14:10:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,F,0,2015,NaT,-134,False
2,16420748,23082241,2018-10-25 22:30:00,2018-10-28 12:58:00,NaT,ELECTIVE,None,HOME,WHITE,NaT,NaT,F,0,2018,NaT,-151,False
3,15206017,25518306,2018-10-22 15:28:00,2018-10-23 19:13:00,NaT,EU OBSERVATION,CLINIC REFERRAL,None,BLACK/AFRICAN AMERICAN,2110-10-22 11:52:00,2110-10-22 17:37:00,F,76,1942,NaT,-92,None
4,14695283,23679175,2018-06-24 18:54:00,2018-06-25 20:15:00,NaT,EU OBSERVATION,PHYSICIAN REFERRAL,None,WHITE,2176-06-24 14:46:00,2176-06-25 20:15:00,F,90,1928,NaT,-158,None


In [4]:
features = [
    AGE,
    SEX,
    "admission_type",
    "admission_location",
    "discharge_location",
    "ethnicity",
]

tab_features = TabularFeatures(
    data=encounters,
    features=features,
    by=ENCOUNTER_ID,
)

In [5]:
tab_features.types

{'admission_location': 'ordinal',
 'sex': 'binary',
 'age': 'numeric',
 'discharge_location': 'ordinal',
 'ethnicity': 'ordinal',
 'admission_type': 'ordinal'}

In [6]:
tab_features.meta["admission_location"].get_mapping()

{0: 'AMBULATORY SURGERY TRANSFER',
 1: 'CLINIC REFERRAL',
 2: 'EMERGENCY ROOM',
 3: 'INFORMATION NOT AVAILABLE',
 4: 'INTERNAL TRANSFER TO OR FROM PSYCH',
 5: 'PACU',
 6: 'PHYSICIAN REFERRAL',
 7: 'PROCEDURE SITE',
 8: 'TRANSFER FROM HOSPITAL',
 9: 'TRANSFER FROM SKILLED NURSING FACILITY',
 10: 'WALK-IN/SELF REFERRAL',
 11: 'nan'}

In [7]:
ordinal_features = tab_features.features_by_type(ORDINAL)
ordinal_features

['admission_location', 'discharge_location', 'ethnicity', 'admission_type']

In [8]:
tab_vectorized = tab_features.vectorize(to_binary_indicators=ordinal_features)

/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].replace(self.meta[feat].get_mapping())
/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[feat] = data[feat].replace(self.meta[feat].get_mapping())
/home/kmckeen/gemini/cyclops/cyclops/processors/feature/feature.py:696: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [9]:
tab_vectorized.shape

(224764, 43)

In [10]:
tab_vectorized.axis_names

['encounter_id', 'features']

In [11]:
tab_vectorized.get_index(FEATURES)

array(['sex', 'age', 'admission_location_AMBULATORY SURGERY TRANSFER',
       'admission_location_CLINIC REFERRAL',
       'admission_location_EMERGENCY ROOM',
       'admission_location_INFORMATION NOT AVAILABLE',
       'admission_location_INTERNAL TRANSFER TO OR FROM PSYCH',
       'admission_location_PACU', 'admission_location_PHYSICIAN REFERRAL',
       'admission_location_PROCEDURE SITE',
       'admission_location_TRANSFER FROM HOSPITAL',
       'admission_location_TRANSFER FROM SKILLED NURSING FACILITY',
       'admission_location_WALK-IN/SELF REFERRAL',
       'discharge_location_ACUTE HOSPITAL',
       'discharge_location_AGAINST ADVICE',
       'discharge_location_ASSISTED LIVING',
       'discharge_location_CHRONIC/LONG TERM ACUTE CARE',
       'discharge_location_DIED',
       'discharge_location_HEALTHCARE FACILITY',
       'discharge_location_HOME', 'discharge_location_HOME HEALTH CARE',
       'discharge_location_HOSPICE', 'discharge_location_OTHER FACILITY',
       'di

In [12]:
numeric_features = tab_features.features_by_type(NUMERIC)
numeric_features

['age']

## Temporal

In [13]:
events = load_dataframe("events.parquet")

events = events.sample(n=int(len(events) / 20))

events.head(5)

2022-07-18 15:28:02,847 INFO cyclops.utils.file - Loading DataFrame from events.parquet


,encounter_id,subject_id,stay_id,event_timestamp,value,event_value,event_value_unit,event_category,event_name
391700,28802821,16439165.0,31283616.0,2018-12-13 21:10:00,Repositioning every 2 hours and PRN,NaN,None,care plans,care plans - altered skin integrity ncp - inte...
158488,29715344,16613914.0,31229998.0,2019-01-16 00:00:00,Non-productive,NaN,None,pulmonary,pulmonary - cough type
308537,29319179,16679905.0,34391014.0,2018-08-12 16:00:00,Lines and tubes concealed,NaN,None,restraint/support systems,restraint/support systems - less restrictive m...
173240,23339961,16529604.0,33927836.0,2015-08-22 08:54:00,None,NaN,None,skin - incisions,skin - incisions - edema location
522647,25642557,16395749.0,31665383.0,2015-01-13 13:28:00,Obtain PT / OT consults to facilitate adaptati...,NaN,None,care plans,care plans - altered sensory perception ncp - ...


In [14]:
# Keep only the most popular events
top_events = events[EVENT_NAME].value_counts()[:100].index

# Force include the target
top_events = np.unique(np.append(top_events, TARGET_NAME))

events = events[events[EVENT_NAME].isin(top_events)]
events.head(5)

,encounter_id,subject_id,stay_id,event_timestamp,value,event_value,event_value_unit,event_category,event_name
158488,29715344,16613914.0,31229998.0,2019-01-16 00:00:00,Non-productive,NaN,None,pulmonary,pulmonary - cough type
308537,29319179,16679905.0,34391014.0,2018-08-12 16:00:00,Lines and tubes concealed,NaN,None,restraint/support systems,restraint/support systems - less restrictive m...
173240,23339961,16529604.0,33927836.0,2015-08-22 08:54:00,None,NaN,None,skin - incisions,skin - incisions - edema location
522647,25642557,16395749.0,31665383.0,2015-01-13 13:28:00,Obtain PT / OT consults to facilitate adaptati...,NaN,None,care plans,care plans - altered sensory perception ncp - ...
161181,21277160,16454910.0,36774255.0,2015-05-30 15:00:00,Endotracheal tube,NaN,None,respiratory,respiratory - o2 delivery device


In [15]:
events[events[EVENT_NAME] == TARGET_NAME].head(5)

,encounter_id,subject_id,stay_id,event_timestamp,value,event_value,event_value_unit,event_category,event_name


In [16]:
aggregator = Aggregator(
    aggfuncs={EVENT_VALUE: MEAN},
    timestamp_col=EVENT_TIMESTAMP,
    time_by=ENCOUNTER_ID,
    agg_by=[ENCOUNTER_ID, EVENT_NAME],
    timestep_size=8,
    window_duration=24,  # Optional
    # agg_meta_for=EVENT_VALUE,  # Optional
)

In [17]:
events = events.reset_index(drop=True)

tmp_features = TemporalFeatures(
    events,
    features=EVENT_VALUE,
    by=[ENCOUNTER_ID, EVENT_NAME],
    timestamp_col=EVENT_TIMESTAMP,
    aggregator=aggregator,
)

In [18]:
aggregated = tmp_features.aggregate()
aggregated.head(5)

2022-07-18 15:28:03,173 INFO cyclops.processors.cleaning - Dropped nulls over columns: event_timestamp. Removed 2 rows.
2022-07-18 15:28:07,063 INFO cyclops.utils.profile - Finished executing function __call__ in 3.895072 s


event_value  \
encounter_id event_name                             timestep                
20068018     pain/sedation - pain assessment method 0                 NaN   
             treatments - turn                      2                 NaN   
                                                    0                 NaN   
             neurological - gcs - eye opening       2                 4.0   
             routine vital signs - ectopy type 1    0                 NaN   

                                                                  timestep_start  
encounter_id event_name                             timestep                      
20068018     pain/sedation - pain assessment method 0        2015-07-21 21:00:00  
             treatments - turn                      2        2015-07-22 13:00:00  
                                                    0        2015-07-21 21:00:00  
             neurological - gcs - eye opening       2        2015-07-22 13:00:00  
             routine vital signs - ectopy type 1    0        2015-07-21 21:00:00

In [19]:
save_dataframe(aggregated, "aggregated.parquet")

2022-07-18 15:28:07,073 INFO cyclops.utils.file - Saving dataframe to aggregated.parquet


'aggregated.parquet'

In [20]:
temp_vectorized = aggregator.vectorize(aggregated)
temp_vectorized.shape

2022-07-18 15:28:08,316 INFO cyclops.utils.profile - Finished executing function vectorize in 1.227015 s


(1, 354, 100, 3)

In [21]:
temp_vectorized.axis_names

['aggfuncs', 'encounter_id', 'event_name', 'timesteps']

## Splitting

Take only the encounters available in all of the datasets and align the datasets over encounters.

In [22]:
tab_vectorized.shape, temp_vectorized.shape

((224764, 43), (1, 354, 100, 3))

In [23]:
tab_vectorized, temp_vectorized = intersect_vectorized(
    [tab_vectorized, temp_vectorized], axes=ENCOUNTER_ID
)

In [24]:
tab_vectorized.shape, temp_vectorized.shape

((354, 43), (1, 354, 100, 3))

### Dataset splits

Split into training, validation, and testing datasets.

Split using the same indices such that the tabular and temporal datasets remain aligned.

In [25]:
tab_splits, temp_splits = split_vectorized(
    [tab_vectorized, temp_vectorized], [0.8, 0.1], axes=ENCOUNTER_ID
)
tab_train_X, tab_val_X, tab_test_X = tab_splits
temp_train, temp_val, temp_test = temp_splits

In [26]:
tab_train_X.shape, tab_val_X.shape, tab_test_X.shape

((283, 43), (36, 43), (35, 43))

In [27]:
temp_train.shape, temp_val.shape, temp_test.shape

((1, 283, 100, 3), (1, 36, 100, 3), (1, 35, 100, 3))

### Split features/targets

Split out the targets in the temporal data.

In [28]:
temp_train_X, temp_train_y = temp_train.split_out(EVENT_NAME, [TARGET_NAME_PROCESSED])
temp_train_X.shape, temp_train_y.shape

((1, 283, 99, 3), (1, 283, 1, 3))

In [29]:
temp_val_X, temp_val_y = temp_val.split_out(EVENT_NAME, [TARGET_NAME_PROCESSED])
temp_val_X.shape, temp_val_y.shape

((1, 36, 99, 3), (1, 36, 1, 3))

In [30]:
temp_test_X, temp_test_y = temp_test.split_out(EVENT_NAME, [TARGET_NAME_PROCESSED])
temp_test_X.shape, temp_test_y.shape

((1, 35, 99, 3), (1, 35, 1, 3))

### Normalization

In [31]:
splits = tab_train_X, tab_val_X, tab_test_X

for split in splits:
    normalizer = VectorizedNormalizer(
        split.get_axis(FEATURES),
        normalizer_map={feat: STANDARD for feat in numeric_features},
    )
    split.add_normalizer(normalizer)
    split.normalize()

tab_train_X, tab_val_X, tab_test_X = splits

In [32]:
# Only normalizing the inputs since the outputs are binary
splits = temp_train_X, temp_val_X, temp_test_X

for split in splits:
    normalizer = VectorizedNormalizer(
        split.get_axis(EVENT_NAME),
        normalization_method=STANDARD,
    )
    split.add_normalizer(normalizer)
    split.normalize()

temp_train_X, temp_val_X, temp_test_X = splits

In [33]:
"Percentage null:", np.isnan(temp_train_X.data).sum() / temp_train_X.data.size

('Percentage null:', 0.9948721609498995)

## Save

In [34]:
tab_train_X.save("tab_train_X.npy")
tab_val_X.save("tab_val_X.npy")
tab_test_X.save("tab_test_X.npy")

temp_train_X.save("temp_train_X.npy")
temp_val_X.save("temp_val_X.npy")
temp_test_X.save("temp_test_X.npy")

temp_train_y.save("temp_train_y.npy")
temp_val_y.save("temp_val_y.npy")
temp_test_y.save("temp_test_y.npy")

2022-07-18 15:28:08,592 INFO cyclops.utils.file - Saving array to tab_train_X.npy
2022-07-18 15:28:08,593 INFO cyclops.utils.file - Saving array to tab_val_X.npy
2022-07-18 15:28:08,594 INFO cyclops.utils.file - Saving array to tab_test_X.npy
2022-07-18 15:28:08,595 INFO cyclops.utils.file - Saving array to temp_train_X.npy
2022-07-18 15:28:08,596 INFO cyclops.utils.file - Saving array to temp_val_X.npy
2022-07-18 15:28:08,597 INFO cyclops.utils.file - Saving array to temp_test_X.npy
2022-07-18 15:28:08,597 INFO cyclops.utils.file - Saving array to temp_train_y.npy
2022-07-18 15:28:08,598 INFO cyclops.utils.file - Saving array to temp_val_y.npy
2022-07-18 15:28:08,599 INFO cyclops.utils.file - Saving array to temp_test_y.npy


'temp_test_y.npy'